# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 16 2023 10:10AM,259433,19,60022081,"GCH Granules USA, Inc.",Released
1,Mar 16 2023 10:10AM,259433,19,60022088,"GCH Granules USA, Inc.",Released
2,Mar 16 2023 10:10AM,259433,19,60022087,"GCH Granules USA, Inc.",Released
3,Mar 16 2023 10:10AM,259433,19,60022086,"GCH Granules USA, Inc.",Released
4,Mar 16 2023 10:10AM,259433,19,60022085,"GCH Granules USA, Inc.",Released
5,Mar 16 2023 10:01AM,259432,19,60022093,"GCH Granules USA, Inc.",Released
6,Mar 16 2023 10:01AM,259432,19,60022089,"GCH Granules USA, Inc.",Released
7,Mar 16 2023 10:01AM,259432,19,60022090,"GCH Granules USA, Inc.",Released
8,Mar 16 2023 10:01AM,259432,19,60022092,"GCH Granules USA, Inc.",Released
9,Mar 16 2023 10:01AM,259432,19,60022091,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,259429,Released,4
11,259430,Released,1
12,259431,Released,3
13,259432,Released,5
14,259433,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259429,NaN,4.0
259430,NaN,1.0
259431,NaN,3.0
259432,NaN,5.0
259433,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259379,1.0,0.0
259384,3.0,33.0
259414,12.0,42.0
259416,0.0,6.0
259420,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259379,1,0
259384,3,33
259414,12,42
259416,0,6
259420,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259379,1,0
1,259384,3,33
2,259414,12,42
3,259416,0,6
4,259420,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259379,1,
1,259384,3,33
2,259414,12,42
3,259416,,6
4,259420,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc."
5,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc."
10,Mar 16 2023 9:57AM,259431,10,"Snap Medical Industries, LLC"
13,Mar 16 2023 9:56AM,259430,20,"HVL, LLC dba Atrium Innovations"
14,Mar 16 2023 9:51AM,259384,15,"Alliance Pharma, Inc."
50,Mar 16 2023 9:21AM,259429,10,Emerginnova
54,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations"
55,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc."
62,Mar 16 2023 8:51AM,259420,10,Hush Hush
82,Mar 16 2023 8:24AM,259416,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",,5
1,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",,5
2,Mar 16 2023 9:57AM,259431,10,"Snap Medical Industries, LLC",,3
3,Mar 16 2023 9:56AM,259430,20,"HVL, LLC dba Atrium Innovations",,1
4,Mar 16 2023 9:51AM,259384,15,"Alliance Pharma, Inc.",3,33
5,Mar 16 2023 9:21AM,259429,10,Emerginnova,,4
6,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",,1
7,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc.",4,3
8,Mar 16 2023 8:51AM,259420,10,Hush Hush,,20
9,Mar 16 2023 8:24AM,259416,19,"AdvaGen Pharma, Ltd",,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",5,
1,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",5,
2,Mar 16 2023 9:57AM,259431,10,"Snap Medical Industries, LLC",3,
3,Mar 16 2023 9:56AM,259430,20,"HVL, LLC dba Atrium Innovations",1,
4,Mar 16 2023 9:51AM,259384,15,"Alliance Pharma, Inc.",33,3
5,Mar 16 2023 9:21AM,259429,10,Emerginnova,4,
6,Mar 16 2023 9:14AM,259427,20,"HVL, LLC dba Atrium Innovations",1,
7,Mar 16 2023 8:55AM,259423,10,"Methapharm, Inc.",3,4
8,Mar 16 2023 8:51AM,259420,10,Hush Hush,20,
9,Mar 16 2023 8:24AM,259416,19,"AdvaGen Pharma, Ltd",6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",5,
1,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",5,
2,Mar 16 2023 9:57AM,259431,10,"Snap Medical Industries, LLC",3,
3,Mar 16 2023 9:56AM,259430,20,"HVL, LLC dba Atrium Innovations",1,
4,Mar 16 2023 9:51AM,259384,15,"Alliance Pharma, Inc.",33,3


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",5.0,NaN
1,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",5.0,NaN
2,Mar 16 2023 9:57AM,259431,10,"Snap Medical Industries, LLC",3.0,NaN
3,Mar 16 2023 9:56AM,259430,20,"HVL, LLC dba Atrium Innovations",1.0,NaN
4,Mar 16 2023 9:51AM,259384,15,"Alliance Pharma, Inc.",33.0,3.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",5.0,0.0
1,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",5.0,0.0
2,Mar 16 2023 9:57AM,259431,10,"Snap Medical Industries, LLC",3.0,0.0
3,Mar 16 2023 9:56AM,259430,20,"HVL, LLC dba Atrium Innovations",1.0,0.0
4,Mar 16 2023 9:51AM,259384,15,"Alliance Pharma, Inc.",33.0,3.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1037703,30.0,4.0
15,259384,33.0,3.0
18,259379,0.0,1.0
19,1037695,58.0,12.0
20,518857,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1037703,30.0,4.0
1,15,259384,33.0,3.0
2,18,259379,0.0,1.0
3,19,1037695,58.0,12.0
4,20,518857,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,30.0,4.0
1,15,33.0,3.0
2,18,0.0,1.0
3,19,58.0,12.0
4,20,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,15,Released,33.0
2,18,Released,0.0
3,19,Released,58.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20
Status,,,,,
Executing,4.0,3.0,1.0,12.0,0.0
Released,30.0,33.0,0.0,58.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20
0,Executing,4.0,3.0,1.0,12.0,0.0
1,Released,30.0,33.0,0.0,58.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20
0,Executing,4.0,3.0,1.0,12.0,0.0
1,Released,30.0,33.0,0.0,58.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()